<a href="https://colab.research.google.com/github/GurramVishalReddy/Parkinson-s-disease-prediction-/blob/main/Parkinson_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import librosa
import gradio as gr
import parselmouth
from parselmouth.praat import call
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)
from imblearn.over_sampling import SMOTE


df = pd.read_csv("/content/Vishu Dataset.csv")
X = df.drop(columns=["name", "status"])
y = df["status"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.25, stratify=y_resampled, random_state=42
)

rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn_bagged = BaggingClassifier(estimator=knn, n_estimators=10, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ("Random Forest", rf),
        ("Bagged KNN", knn_bagged)
    ],
    voting='soft'
)

voting_clf.fit(X_train, y_train)


def extract_22_features(file_path):
    y, sr = librosa.load(file_path)

    duration = librosa.get_duration(y=y, sr=sr)
    amplitude = np.max(np.abs(y)) if len(y) > 0 else 0
    print(f"Audio Duration: {duration}s, Max Amplitude: {amplitude}")

    snd = parselmouth.Sound(file_path)
    if snd.duration < 0.3:
        raise ValueError("Audio too short. Please upload a longer voice sample.")

    point_process = call(snd, "To PointProcess (periodic, cc)", 75, 500)
    pitch = call(snd, "To Pitch", 0.0, 75, 600)

    try:
        mdvp_fo = pitch.get_mean() if pitch.get_mean() > 0 else np.nan
        mdvp_fhi = pitch.get_maximum() if pitch.get_maximum() > 0 else np.nan
        mdvp_flo = pitch.get_minimum() if pitch.get_minimum() > 0 else np.nan
        jitter_local = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        jitter_abs = call(point_process, "Get jitter (absolute)", 0, 0, 0.0001, 0.02, 1.3)
        mdvp_rap = call(point_process, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
        mdvp_ppq = call(point_process, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
        jitter_ddp = call(point_process, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
        shimmer_local = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        mdvp_shimmer = shimmer_local * 100 if shimmer_local > 0 else np.nan
        mdvp_shimmer_db = call([snd, point_process], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        shimmer_apq3 = call([snd, point_process], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        shimmer_apq5 = call([snd, point_process], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        mdvp_apq = call([snd, point_process], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        shimmer_dda = call([snd, point_process], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        nhr = call(snd, "Get noise to harmonics ratio (HNR)", 0.0, 75, 600, 0.0, 0.0, "mean")
        hnr = call(snd, "Get harmonicity (cc)", 0.01, 75, 0.1, 1.0).get_value(1)
        print(f"Raw Features: Fo={mdvp_fo}, Fhi={mdvp_fhi}, Jitter={jitter_local}, Shimmer={shimmer_local}, NHR={nhr}")
    except Exception as e:
        mdvp_fo = mdvp_fhi = mdvp_flo = jitter_local = jitter_abs = mdvp_rap = mdvp_ppq = jitter_ddp = np.nan
        shimmer_local = mdvp_shimmer = mdvp_shimmer_db = shimmer_apq3 = shimmer_apq5 = mdvp_apq = shimmer_dda = np.nan
        nhr = hnr = np.nan
        print(f"Feature extraction error: {str(e)}")


    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y).mean()


    feature_vector = np.array([
        mdvp_fo, mdvp_fhi, mdvp_flo,
        jitter_local, jitter_abs,
        mdvp_rap, mdvp_ppq, jitter_ddp,
        mdvp_shimmer, mdvp_shimmer_db, shimmer_apq3, shimmer_apq5,
        mdvp_apq, shimmer_dda,
        nhr, hnr,
        np.nan, np.nan,
        np.nan, np.nan, np.nan,
        np.nan
    ])


    X_train_df = pd.DataFrame(X_train, columns=X.columns)
    for i, col in enumerate(X.columns):
        if np.isnan(feature_vector[i]):
            feature_vector[i] = X_train_df[col].mean()

    return dict(zip(X.columns, feature_vector))

def predict_from_voice_file(audio_file):
    try:

        features = extract_22_features(audio_file)

        features_df = pd.DataFrame([features], columns=X.columns)
        features_scaled = scaler.transform(features_df)

        proba = voting_clf.predict_proba(features_scaled)[0, 1]
        prediction = 1 if proba >= 0.5 else 0

        prediction_text = "🧠 Parkinson's Detected" if prediction == 1 else "✅ No Parkinson's Detected"

        metrics = {
            "Accuracy": accuracy_score(y_test, voting_clf.predict(X_test)),
            "Precision": precision_score(y_test, voting_clf.predict(X_test)),
            "Recall": recall_score(y_test, voting_clf.predict(X_test)),
            "F1 Score": f1_score(y_test, voting_clf.predict(X_test)),
            "ROC AUC": roc_auc_score(y_test, voting_clf.predict_proba(X_test)[:,1]),
        }
        metrics_str = "\n".join([f"{k}: {v:.2f}" for k, v in metrics.items()])


        features_str = "\n".join([f"{k}: {v:.4f}" for k, v in features.items()])


        output = f"{prediction_text}\n\n📊 Extracted Features:\n{features_str}\n\n📊 Model Metrics:\n{metrics_str}"

        return output

    except Exception as e:
        return f"❌ Error processing the file: {str(e)}"


interface = gr.Interface(
    fn=predict_from_voice_file,
    inputs=gr.Audio(type="filepath", label="Upload Voice Sample (.wav)"),
    outputs="text",
    title="Parkinson's Disease Prediction",
    description="This app predicts Parkinson’s Disease from voice by extracting 22 biomedical features from your audio."
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93d63427100d7a2217.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
